In [158]:
""" Imports """
import csv
from math import floor, ceil
import matplotlib.pyplot as plt
from DPOMDP_Writer.Reader import *
from DPOMDP_Writer.DPOMDPWriterMedium import generate_safety_scenarios

## The following code reads the outputs from the MADP Solver and reasons about their values

In [159]:
""" Read policy and return simplified policy dictionary """
from genericpath import exists


def exists_in_dict(dict, elem):
    # Checks if an element is in a dict. 
    # If yes, return its key
    for x in dict.keys():
        if dict[x] == elem:
            return [True, x]
    return [False, -1]


    
def get_policy_from_tree(tree):
    [nodes,edges] = tree.split(";")
    nodes=nodes.split(">")[:-1]
    edges=edges.split("-")[:-1]
    #print(nodes)
    #print(edges)
    level0 = nodes[0]
    level1 = nodes[1:]
    if len(edges) != len(level1):
        print("ERROR!!")
    policy_dict = {}
    for x in range(len(edges)):
        elem = edges[x]
        obs = elem.split(",")[2]
        # print(obs)
        node = level1[x]
        action = node.split(",")[1]
        #action = analyze_action(action)
        [exists, key] = exists_in_dict(policy_dict, action)
        # print(exists)
        if exists:
            new_key = key + " + " + obs
            del policy_dict[key]
            policy_dict[new_key] = action
        else:
            policy_dict[obs] = action
    #print(policy_dict)
    return[level0, policy_dict]    

In [160]:
""" Making TEX tables """
def abbreviate_sub(sub_str):
    sc = ("speedcontrol" in sub_str)
    foll = ("following" in sub_str)
    err = ("error" in sub_str)
    hold = ("hold" in sub_str)
    stby = ("standby" in sub_str)
    ovr = ("override" in sub_str)
    if sc:
        return "\SC"
    if hold:
        return "\hold"
    if err:
        return "\Err"
    if foll:
        return "\Foll"
    if stby:
        return "\Stby"
    if ovr:
        return "\Ovr"
    else:
        return sub_str
    
def abbreviate_long(long_str):
    fsch = ("'following', 'speedcontrol', 'hold'" in long_str)
    
    if fsch:
        return "\FSCH"
    else:
        if "," in long_str:
            substring_arr = long_str.split(",")
            output_str = ""
            for ch_str in substring_arr:
                output_str += abbreviate_sub(ch_str) + ","
            return output_str[:-1]
        else:
            return abbreviate_sub(long_str)
    

def make_table(header: list, contents:list, caption: str, label: str):
    col_center = "c " * len(header)
    table_prefix = "\\begin{table}[]\n\centering\n\\begin{tabular}{" + col_center[:-1] + "}\n\\toprule\n"
    table_suffix = f"\\bottomrule\n\end{{tabular}}\n\caption{{{caption}}}\n\label{{tab:{label}}}\n\end{{table}}"
    table_str = table_prefix
    #append header
    header_str = ""
    for elem in header:
        header_str += str(elem) + " & "
    table_str += header_str[:-3] + "\\\\\n"
    table_str += "\\toprule\n"
    #append the contents
    for entry in contents:
        entry_str = ""
        for elem in entry:
            entry_str += abbreviate_long(str(elem)) + " & "
        table_str += entry_str[:-3] + "\\\\\n"
    table_str += table_suffix
    return table_str

In [161]:
""" Return transition next-modes if there is communication"""
def transitions(mode, scenario):
    t_file = f"figs/ACC{mode}-scen{scenario}_mach"
    policy = read_unbalanced_tree(t_file,generate_safety_scenarios()[scenario])
    [nodes,edges] = policy.split(";")
    nodes = nodes.split(">")
    n = nodes[0]
    [name, action] = n.split(",")
    end_modes = []
    if "-communicate" in action:
        edge_list = edges.split("-")[:-1]
        for edge in edge_list:
            [start,end,end_mode] = edge.split(",")
            end_modes.append(end_mode)
    return end_modes

#print(transitions("standby",1))

In [162]:

#tree_values_s0-ACC.csv

def get_tree_value(scenario,mode):
    tree_file = f"results/tree_values_s{scenario}-ACC.csv"
    with open(tree_file, newline='') as csvfile:
        t_reader = csv.reader(csvfile)
        for row in t_reader:
            if row[0]== mode:
                return float(row[1])

modes = ["standby", "following", "speedcontrol", "error", "hold", "override", "off"]
# for mode in modes:
#     for scenario in range(0,8):
#         value = get_tree_value(scenario,mode)

"""Finding which scenarios required communication"""

def human_actions_summary(mode, scenario):
    """Returns booleans on whether certain actions were taken"""
    t_file = f"figs/ACC{mode}-scen{scenario}_hum"
    policy = read_unbalanced_tree(t_file,generate_safety_scenarios()[scenario])
    if "safest" in policy:
        physical = True
    else:
        physical = False
    return physical

def machine_actions_summary(mode, scenario):
    """Returns booleans on whether certain actions were taken"""
    t_file = f"figs/ACC{mode}-scen{scenario}_mach"
    policy = read_unbalanced_tree(t_file,generate_safety_scenarios()[scenario])
    #print(policy)
    if "-comm" in policy:
        comm = True
    else:
        comm = False
    return comm

def safety_analysis(mode, scenario):
    """ Returns TRUE if accident was avoided """
    mach_t_file = f"figs/ACC{mode}-scen{scenario}_mach"
    hum_t_file = f"figs/ACC{mode}-scen{scenario}_hum"
    h_tree = read_unbalanced_tree(hum_t_file,generate_safety_scenarios()[scenario])
    [nodes,edges] = h_tree.split(";")
    h_nodes=nodes.split(">")[:-1]
    h_edges=edges.split("-")[:-1]
    h_level0 = h_nodes[0]
    h_level1 = h_nodes[1:]
    m_tree = read_unbalanced_tree(mach_t_file,generate_safety_scenarios()[scenario])
    [nodes,edges] = m_tree.split(";")
    nodes=nodes.split(">")[:-1]
    edges=edges.split("-")[:-1]
    m_level0 = nodes[0]
    m_level1 = nodes[1:]
    #print([h_level0,h_level1,m_level0,m_level1])
    hum_safe_init_action = "safest" in h_level0
    mach_safe_init_action = "safest" in m_level0
    #If the first action is a safe one:
    if hum_safe_init_action | mach_safe_init_action:
        #see if the second actions are safe:
        for i in range(len(h_level1)):
            m_safe = "safest" in m_level1[i]
            #print(m_safe)
            h_safe = "safest" in h_level1[i]
            safe = (h_safe | m_safe)
            if (safe==False):
                return False
        return True
    else:
        return False
    


safe_communication = []
safe_hum_action = []
safe_no_comm = []
unsafe_no_comm = []
for mode in modes:
    for scenario in range(0,8):
        #True means machine had to update the interface
        comm = machine_actions_summary(mode,scenario)
        safe = safety_analysis(mode,scenario)
        hum_moves = human_actions_summary(mode,scenario)
        if (comm & safe):
            safe_communication.append([scenario,mode])
        if (safe & hum_moves):
            safe_hum_action.append([mode,scenario])
        elif safe:
            safe_no_comm.append([mode,scenario])
        else:
            unsafe_no_comm.append([mode,scenario])
# print(f"Interface Update for safety: {safe_communication}\n")
# print(f"Safe, no interface update or human action needed {safe_no_comm}\n")
# print(f"Unsafe always: {unsafe_no_comm}\n")
# print(f"Human takes action for safety: {safe_hum_action}")

"""Make tables out of interface updates"""

header = ["Scenario","Mode"]
caption = "Scenario, Mode pairings where the machine will communicate the mode to the user"
label = "interface_policy"

#by mode
table_contents = []
for mode in modes:
    entry_str = ""
    for elem in safe_communication:
        if elem[1] == mode:
            entry_str += str(elem[0])
    table_contents.append([mode,entry_str])


header = ["Mode", "Scenario"]
caption = "Scenario, Mode pairings where the machine will communicate the mode to the user, organized by mode"
label = "if_pol_by_mode"
interface_table = make_table(header, table_contents, caption, label)
#print(interface_table)

#by scenario
table_contents = []
for scenario in range(0,8):
    entry_str = ""
    for elem in safe_communication:
        #print(elem[0])
        if elem[0] == scenario:
            entry_str += "\\" + str(elem[1]) + ", "
    table_contents.append([str(scenario),entry_str[:-2]+" "])
    
    
#by mode
table_contents = []
for mode in modes:
    entry_str = ""
    for elem in safe_hum_action:
        #print(elem)
        if elem[0] == mode:
            entry_str += str(elem[1])
    table_contents.append([mode,entry_str])


header = ["Mode", "Scenario"]
caption = "Scenario, Mode pairings where the user needs to take action, organized by mode"
label = "if_pol_by_mode"
#print(table_contents)
interface_table = make_table(header, table_contents, caption, label)
#print(interface_table)

# #by scenario
# table_contents = []
# for scenario in range(0,8):
#     entry_str = ""
#     for elem in safe_communication:
#         #print(elem[0])
#         if elem[0] == scenario:
#             entry_str += "\\" + str(elem[1]) + ", "
#     table_contents.append([str(scenario),entry_str[:-2]+" "])


# header = ["Scenario","Mode"]
# caption = "Scenario, Mode pairings where the machine will communicate the mode to the user, organized by scenario"
# label = "if_pol_by_scen"
# interface_table = make_table(header, table_contents, caption, label)
# #print(interface_table)

# """ Make table of values, starting from least to greatest """
# value_dict = {}
# for mode in modes:
#     for scenario in range(0,8):
#         key = get_tree_value(scenario,mode)
#         value = f"{mode}-{scenario}"
#         if key in value_dict:
#             new_str = value_dict[key] + ", " + value
#         else: 
#             new_str = value
#         value_dict[key] = new_str
# #print sorted table
# output = "\\begin{table}[]\n"
# output += "\\centering\n"
# output += "\\begin{tabular}{c c c}\n"
# output += "\\toprule\n"
# output += "Tree & Applicable Scenarios & Policy  \\\\ \n"
# output += "\\toprule\n"
# for key in sorted(value_dict.keys()):
#     output += f"{value_dict[key]} & {key}\\\\ \n"
# output += "\\bottomrule"
# output += "\\end{tabular}\n"
# output += "\\caption{" + caption + "}\n"
# output += "\\label{tab:my_label}\n"
# output += "\\end{table}\n"
# output += "\n\n"
# #print(output)

table_contents = []
for elem in safe_communication:
    [scenario,mode] = elem
    table_contents.append([scenario,mode,transitions(mode, scenario)])

header = ["Scenario", "Start Mode", "End Mode(s)"]
caption = "Transitions where the machine communicates"
label = "comm_trans"
#print(table_contents)
interface_table = make_table(header, table_contents, caption, label)
#print(interface_table)

f = open("tex_output.tex","w")
f.writelines(interface_table)
f.close()

""" Method: check if there's an entry where a is in row 1 and b is in row 2"""
def find_entry(a, b, check_table):
    for line in check_table:
        if type(line) == str:
            line = line.split(",")
        a_val = line[1]
        b_val = line[2]
        #print(a_val)
        #print(b_val)
        if (a in a_val) or (a_val in a):
            if type(b_val) == list:
                for bv in b_val:
                    #print(b)
                    #print(bv)
                    if (bv in b) or (b in bv):
                        #print(f"{b},{b_val}")
                        return True
            else:
                if (b_val in b) or (b in b_val):
                        #print(f"{b},{b_val}")
                        return True
    return False
                
def find_entry_mc_table(a, b, check_table):
    for line in check_table:
        [a_val,expected,b_val] = line.split(",")
        if (a in a_val) or (a_val in a):
            if (b_val in b) or (b in b_val):
                        #print(f"{b},{b_val}")
                        return True
    return False   
        

""" Compare to model checking results """
g = open("mc_results.csv", "r")
lines = g.readlines()
g.close
missing_trans = []
for line in lines:
    found = False
    [start,expected,next] = line.split(",") 
    contains = find_entry(start,next,table_contents)
    if contains == False:
        missing_entry = [start,next[:-1]]
        #print(missing_entry)
        missing_trans.append(missing_entry)
#print(f"Missing from dpomdp: {missing_trans}")

#res = []
#[res.append(x) for x in missing_trans if x not in res]

for elem in missing_trans:
    print(f"{elem[0]} & & {elem[1]}\\\\")

""" See if mc missed any trans we found """
mc_missed = []
for entry in table_contents:
    [num,start,next] = entry
    for elem in next:
        contains = find_entry_mc_table(start,elem,lines)
        if contains == False:
            missing = [start,elem]
            mc_missed.append(missing)

res = []
[res.append(x) for x in mc_missed if x not in res]
#print(f"Missing from model checker:{res}")



#print(len(lines))


 
    
        



following & & standby\\
following & & override\\
following & & hold\\
speedcontrol & & standby\\
hold & & hold\\
hold & & off\\
hold & & off\\
hold & & off\\
hold & & hold\\
hold & & off\\


[None, None]

In [163]:
"""Make a table that shows [scenario, start mode] pairings for all times the machine updates the interface """
mach_policy_table = []
#hum_policy_table = []
modes = ["standby", "following", "speedcontrol", "error", "hold", "override","off"]

for mode in modes:
    for scenario in range(0,8):
        mach_t_file = f"figs/ACC{mode}-scen{scenario}_mach"
        #print(mach_t_file)
        #hum_t_file = f"figs/ACC{mode}-scen{scenario}_hum"
        #[h_level1, h_level2] = get_policy_from_tree(read_unbalanced_tree(hum_t_file,generate_safety_scenarios()[scenario]))
        [mlevel1,mlevel2] = get_policy_from_tree(read_unbalanced_tree(mach_t_file,generate_safety_scenarios()[scenario]))
        action = mlevel1.split(",")[1]
        if "dontcomm" in action:
            comm = "No"
        else:
            mach_policy_table.append([scenario, mode])
        for elem in mlevel2.keys():
            start_mode = elem
            action = mlevel2[elem]
            if "dontcomm" in action:
                comm = "No"
            else:
                mach_policy_table.append([scenario, start_mode])
print(mach_policy_table)

[[3, 'standby'], [5, 'standby'], [6, 'standby'], [7, 'standby'], [1, 'following'], [2, 'following'], [3, 'following'], [4, 'following'], [5, 'following'], [6, 'following'], [7, 'following'], [3, 'speedcontrol'], [5, 'speedcontrol'], [6, 'speedcontrol'], [7, 'speedcontrol'], [0, 'standby'], [1, 'standby'], [2, 'standby'], [3, 'standby'], [5, 'standby'], [6, 'standby'], [3, 'hold'], [5, 'hold'], [6, 'hold'], [7, 'hold'], [1, 'override'], [2, 'override'], [3, 'override'], [4, 'override'], [5, 'override'], [6, 'override'], [7, 'override'], [0, 'off'], [1, 'off'], [2, 'off'], [3, 'off'], [5, 'off'], [6, 'off']]


## The code below works to merge identical joint policies to make it easier to view policy trees

In [164]:

def check_tree(tree_file1, instance_name, mergefile):
    """ Read through to see if mach + hum trees exist in file """
    """ If policies exist, add this instance name to those policies """
    """ If not, add policy to tree"""
    scenario = int(instance_name[-1])
    trees = open(mergefile, "r")
    reader = trees.readlines()
    trees.close()
    output_lines = ""
    tree_str1 = read_unbalanced_tree(tree_file1,generate_safety_scenarios()[scenario])
    #print(tree_str1)
    new_elem = True
    for elem in reader:
        #print(elem)
        [instance, tree1, newline] = elem.split(":")
        if (tree_str1 == tree1):
            new_instance_name = instance + " + " + instance_name
            output_lines += new_instance_name + ":" + tree1 + ":" + "\n"
            new_elem = False
        else:
            output_lines += elem
    if new_elem:
        #print("Adding to output")
        output_lines += instance_name + ":" + tree_str1 + ":" + "\n"
        #print(output_lines)
    #print(new_elem)
    return(output_lines)


In [165]:
def merge_trees(prefix, modes,agent):
    '''Search all results and combine identical policy trees'''
    if agent == "human":
        suf = "_hum"
    elif agent == "machine":
        suf = "_mach"
    else:
        print("Error! Wrong agent type.")
    merge_name = agent + "merge.csv"
    writer = open(merge_name, "w")
    writer.close()
    for mode in modes:
        for scenario in range(8):
            instance_name = mode + str(scenario)
            #print("Instance Name: " + instance_name)
            filename1 = "figs/" + prefix + mode + "-scen" + str(scenario) + suf
            output = check_tree(filename1, instance_name, merge_name)
            writer = open(merge_name, "w")
            writer.writelines(output)
            writer.close()
            


In [166]:
def make_fig(figname, caption, label):
    """ Output tex fig as string """
    output = ""
    output+="\\begin{figure}\n"
    output+="\\centering\n"
    output+="\\includegraphics[\\textwidth]{\n"
    output+= figname + "}\n"
    output+="\\caption{" + caption + "}\n"
    output+="\\label{fig:" + label + "}\n"
    output += "\\end{figure}\n"
    return output

In [167]:
modes = ["standby", "following", "speedcontrol", "error", "hold", "override"]
merge_trees("ACC", modes, "machine")
#merge_trees("ACC", modes, "human")

In [168]:
def shorten_str(inp_str):
    if " + " in inp_str:
        output = ""
        for elem in inp_str.split(" + "):
            output += shorten_str(elem) + "+"
        return output[:-1]
    if inp_str == "following":
        return "\\Foll"
    if inp_str == "following + speedcontrol":
        return "\\FSC"
    if inp_str == "speedcontrol":
        return "\\SC"
    if inp_str == "dontcommunicate":
        return "dont comm"
    if inp_str == "communicate":
        return "comm"
    if inp_str == "dontpushbutton":
        return "dont push"
    if inp_str == "pushbutton":
        return "push"
    if inp_str == "error":
        return "\\Err"
    if inp_str == "standby":
        return "\\Stby"
    if inp_str == "override":
        return "\\OVR"
    if inp_str == "hold":
        return "\\hold"
    return inp_str 


def get_action_str(action):
    phys = action.split("-")[0][1:]
    comm = action.split("-")[1][:-1]
    if phys != "none":
        line = shorten_str(phys) + " + " + shorten_str(comm)
    else:
        line = shorten_str(comm) 
    return line

def read_policy(policy):
    """ Takes output from get_policy_from_tree and makes it more readable """
    [line1,line2] = policy
    init_action = line1.split(",")[1]
    line1_str = get_action_str(init_action)
    #print(line1_str)
    line2_str = ""
    line3_str = ""
    if len(line2.keys()) <= 2:
        for obs in line2.keys():
            line2_str += shorten_str(obs) + ": " + get_action_str(line2[obs]) + "; "
    else:
        counter = 0
        for obs in line2.keys():
            if counter < 2:
                line2_str += shorten_str(obs) + ": " + get_action_str(line2[obs]) + "; "
            else: 
                line3_str += shorten_str(obs) + ": " + get_action_str(line2[obs]) + "; "
            counter += 1
    return [line1_str,line2_str[:-2], line3_str[:-2]]
        
    

In [169]:
def combine_range(inp_list):
    """ combines consecutive numbers in a list 
    e.g. [1,2,3] becomes [1-3] """
    consecutive = False
    range_str = ""
    for x in range(0,len(inp_list)):
        if consecutive:
            #check if last elem
            if (x == len(inp_list) -1):
                last = inp_list[x]
                range_str += str(first) + "-" + str(last)
            #check if still consecutive
            elif inp_list[x] == inp_list[x+1] -1:
                last = inp_list[x+1]
            else:
                range_str += str(first) + "-" + str(last) + "," 
                consecutive = False
        elif (x == len(inp_list) -1):
                #we've reached last elem
                range_str += str(inp_list[x])
        elif inp_list[x] == inp_list[x+1] - 1:
            # number is consecutive, so track it
            consecutive = True
            first = inp_list[x]
            last = inp_list[x+1]
        else:
            #consecutive = False
            range_str += str(inp_list[x]) + ","
    return range_str
    
def combine_scens(scens_list):
    #first, combine by mode
    s_dict = {}
    ret_list = []
    for s in scens_list:
        mode = s[:-1]
        scen = int(s[-1])
        if mode in s_dict.keys():
            temp = s_dict[mode]
            temp.append(scen)
            s_dict[mode] = temp
        else:
            s_dict[mode] = [scen]
    # then, combine by scenario range
    for key in s_dict.keys():
        range_str = combine_range(s_dict[key])
        ret_list.append("\\" + key + "{} " + range_str)
    print(ret_list)
    return ret_list

def list_to_string(inp_list):
    output_str = ""
    for elem in inp_list:
        output_str += str(elem) + " + "
    return output_str[:-2]

def make_table_entry(tree_number, l1, num_per_row):
    [scens, tree, newline] = l1.split(":")
    [hline1,hline2,hline3] = read_policy(get_policy_from_tree(tree))
    
    # print("Human tree:")
    # print(hline1)
    # print(hline2)
    # print("Machine tree:")
    # print(mline1)
    # print(mline2)
    
    l = combine_scens(scens.split(" + "))
    print(l)
    
    num_row = ceil(len(l)/num_per_row)
    i = 0
    if num_row > 1:
        if num_row == 2:
            size = num_row + 1
        else: 
            size = num_row
        output = "\\multirow{" + str(size) + "}{*}{" + str(tree_number) + "} & "
        while len(l) > 0:
            if len(l) > num_per_row:
                line = list_to_string(l[:num_per_row])
                l = l[num_per_row:]
            elif len(l) == num_per_row:
                line = list_to_string(l[:num_per_row])
                l = []
            else:
                line = list_to_string(l)
                l = []
            if (i == 0):
                #top line
                output += line + " + & " + hline1 +  "\\\\\n"
            elif (i == 1) & (i != num_row - 1):
                #secondline but not last
                output += " & " + line + " + & " + hline2 + "\\\\\n"
            elif (i == 1):
                #second and last line
                output += " & " + line + " & " + hline2 +  "\\\\\n"
            elif (i == 2) & (i != num_row - 1):
                #thirdline but not last
                output += " & " + line + " + & " + hline3 + "\\\\\n"
            elif (i == 1):
                #third and last line
                output += " & " + line + " & " + hline3 +  "\\\\\n"
            elif i != num_row - 1: 
                #middle lines
                output += " & " + line + " + & \\\\\n"
            else:
                #last_line
                output += " & " + line + " & \\\\\n"
            i += 1
    else:
        output = "\\multirow{3}{*}{" + str(tree_number) + "} & " 
        output += "\\multirow{3}{*}{" + list_to_string(l) + "} & "
        output += hline1 +  "\\\\\n"
        output += "& & " + hline2 + "\\\\\n"
        output += "& & " + hline3 + "\\\\\n"
    #if num_row == 2:
        #handles case where we need a blank line
        #output += "& & " + hline3 + "\\\\\n"
    return output

def start_table_str():
    output = "\\begin{table}[]\n"
    output += "\\centering\n"
    output += "\\begin{tabular}{c c c}\n"
    output += "\\toprule\n"
    output += "Tree & Applicable Scenarios & Policy  \\\\ \n"
    output += "\\toprule\n"
    return output
    
def end_table_str(caption):
    output = "\\bottomrule"
    output += "\\end{tabular}\n"
    output += "\\caption{" + caption + "}\n"
    output += "\\label{tab:my_label}\n"
    output += "\\end{table}\n"
    output += "\n\n"
    return output

In [170]:
def get_tex_table_from_mergefile(merge_name,caption):
    f = open("machinemerge.csv", "r")
    lines = f.readlines()
    f.close()
    tree_number = 0
    output = start_table_str()
    for l in lines:
        output += make_table_entry(tree_number, l, num_per_row=1)
        if tree_number == 10:
            output += end_table_str(caption)
            output += start_table_str()
        else:
            output += "\\midrule\\\\\n"
        tree_number += 1
    output += end_table_str(caption)
    return output

caption = "Human Policies"
g = open("humantable.tex", "w")
g.writelines(get_tex_table_from_mergefile("humanmerge.csv", caption))
g.close()

caption = "Machine Policies"
g = open("machinetable.tex", "w")
g.writelines(get_tex_table_from_mergefile("machinemerge.csv", caption))
g.close()

    

['\\standby{} 0']
['\\standby{} 0']
['\\standby{} 1-2,4']
['\\standby{} 1-2,4']
['\\standby{} 3,5-7']
['\\standby{} 3,5-7']
['\\following{} 0']
['\\following{} 0']
['\\following{} 1,4']
['\\following{} 1,4']
['\\following{} 2']
['\\following{} 2']
['\\following{} 3,5-7', '\\speedcontrol{} 3,5-7']
['\\following{} 3,5-7', '\\speedcontrol{} 3,5-7']
['\\speedcontrol{} 0']
['\\speedcontrol{} 0']
['\\speedcontrol{} 1-2,4']
['\\speedcontrol{} 1-2,4']
['\\error{} 0,2-3,6']
['\\error{} 0,2-3,6']
['\\error{} 1,5']
['\\error{} 1,5']
['\\error{} 4,7']
['\\error{} 4,7']
['\\hold{} 0']
['\\hold{} 0']
['\\hold{} 1']
['\\hold{} 1']
['\\hold{} 2,4']
['\\hold{} 2,4']
['\\hold{} 3,5-7']
['\\hold{} 3,5-7']
['\\override{} 0']
['\\override{} 0']
['\\override{} 1,4-5']
['\\override{} 1,4-5']
['\\override{} 2']
['\\override{} 2']
['\\override{} 3']
['\\override{} 3']
['\\override{} 6-7']
['\\override{} 6-7']
['\\standby{} 0']
['\\standby{} 0']
['\\standby{} 1-2,4']
['\\standby{} 1-2,4']
['\\standby{} 3,5-7']
